In [7]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import json
import munch
import logging
import os
import json
import sys
import numpy as np
from scipy import optimize
import tensorflow as tf
import tensorflow_probability as tfp
import time
import munch
workdir = os.getcwd()+"/"
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.get_device_details(gpus[0])
sys.path.insert(0, workdir+'/DeepBSDE')

from plotly.subplots import make_subplots
import plotly.graph_objects as go
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=200)
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
pd.options.display.float_format = '{:.3g}'.format
sns.set(font_scale = 1.0, rc={"grid.linewidth": 1,'grid.color': '#b0b0b0', 'axes.edgecolor': 'black',"lines.linewidth": 3.0}, style = 'whitegrid')

# import DGM

Num GPUs Available:  1


In [8]:
class LSTMLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, trans1 = "tanh", trans2 = "tanh"):
        '''
        Args:
            input_dim (int):       dimensionality of input data
            output_dim (int):      number of outputs for LSTM layers
            trans1, trans2 (str):  activation functions used inside the layer; 
                                   one of: "tanh" (default), "relu" or "sigmoid"
        
        Returns: customized Keras layer object used as intermediate layers in DGM
        '''        
        
        # create an instance of a Layer object (call initialize function of superclass of LSTMLayer)
        super(LSTMLayer, self).__init__()
        
        # add properties for layer including activation functions used inside the layer  
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        if trans1 == "tanh":
            self.trans1 = tf.nn.tanh
        elif trans1 == "relu":
            self.trans1 = tf.nn.relu
        elif trans1 == "sigmoid":
            self.trans1 = tf.nn.sigmoid
        
        if trans2 == "tanh":
            self.trans2 = tf.nn.tanh
        elif trans2 == "relu":
            self.trans2 = tf.nn.relu
        elif trans2 == "sigmoid":
            self.trans2 = tf.nn.relu
        
        ### define LSTM layer parameters (use Xavier initialization)
        # u vectors (weighting vectors for inputs original inputs x)
        self.Uz = self.add_variable("Uz", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        self.Ug = self.add_variable("Ug", shape=[self.input_dim ,self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        self.Ur = self.add_variable("Ur", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        self.Uh = self.add_variable("Uh", shape=[self.input_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        
        # w vectors (weighting vectors for output of previous layer)        
        self.Wz = self.add_variable("Wz", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        self.Wg = self.add_variable("Wg", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        self.Wr = self.add_variable("Wr", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        self.Wh = self.add_variable("Wh", shape=[self.output_dim, self.output_dim],
                                    initializer = tf.keras.initializers.glorot_normal)
        
        # bias vectors
        self.bz = self.add_variable("bz", shape=[1, self.output_dim])
        self.bg = self.add_variable("bg", shape=[1, self.output_dim])
        self.br = self.add_variable("br", shape=[1, self.output_dim])
        self.bh = self.add_variable("bh", shape=[1, self.output_dim])
    
    
    # main function to be called 
    def call(self, S, X):
        '''Compute output of a LSTMLayer for a given inputs S,X .    

        Args:            
            S: output of previous layer
            X: data input
        
        Returns: customized Keras layer object used as intermediate layers in DGM
        '''   
        
        # compute components of LSTM layer output (note H uses a separate activation function)
        Z = self.trans1(tf.add(tf.add(tf.matmul(X,self.Uz), tf.matmul(S,self.Wz)), self.bz))
        G = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ug), tf.matmul(S, self.Wg)), self.bg))
        R = self.trans1(tf.add(tf.add(tf.matmul(X,self.Ur), tf.matmul(S, self.Wr)), self.br))
        
        H = self.trans2(tf.add(tf.add(tf.matmul(X,self.Uh), tf.matmul(tf.multiply(S, R), self.Wh)), self.bh))
        
        # compute LSTM layer output
        S_new = tf.add(tf.multiply(tf.subtract(tf.ones_like(G), G), H), tf.multiply(Z,S))
        
        return S_new

#%% Fully connected (dense) layer - modification of Keras layer class
   
class DenseLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, transformation=None):
        '''
        Args:
            input_dim:       dimensionality of input data
            output_dim:      number of outputs for dense layer
            transformation:  activation function used inside the layer; using
                             None is equivalent to the identity map 
        
        Returns: customized Keras (fully connected) layer object 
        '''        
        
        # create an instance of a Layer object (call initialize function of superclass of DenseLayer)
        super(DenseLayer,self).__init__()
        self.output_dim = output_dim
        self.input_dim = input_dim
        
        ### define dense layer parameters (use Xavier initialization)
        # w vectors (weighting vectors for output of previous layer)
        self.W = self.add_variable("W", shape=[self.input_dim, self.output_dim],
                                   initializer = tf.keras.initializers.glorot_normal)
        
        # bias vectors
        self.b = self.add_variable("b", shape=[1, self.output_dim])
        
        if transformation:
            if transformation == "tanh":
                self.transformation = tf.tanh
            elif transformation == "relu":
                self.transformation = tf.nn.relu
        else:
            self.transformation = transformation
    
    
    # main function to be called 
    def call(self,X):
        '''Compute output of a dense layer for a given input X 

        Args:                        
            X: input to layer            
        '''
        
        # compute dense layer output
        S = tf.add(tf.matmul(X, self.W), self.b)
                
        if self.transformation:
            S = self.transformation(S)
        
        return S

#%% Neural network architecture used in DGM - modification of Keras Model class
    
class DGMNet(tf.keras.Model):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, layer_width, n_layers, input_dim, final_trans=None):
        '''
        Args:
            layer_width: 
            n_layers:    number of intermediate LSTM layers
            input_dim:   spaital dimension of input data (EXCLUDES time dimension)
            final_trans: transformation used in final layer
        
        Returns: customized Keras model object representing DGM neural network
        '''  
        
        # create an instance of a Model object (call initialize function of superclass of DGMNet)
        super(DGMNet,self).__init__()
        
        # define initial layer as fully connected 
        # NOTE: to account for time inputs we use input_dim+1 as the input dimensionality
#         self.initial_layer = DenseLayer(layer_width, input_dim+1, transformation = "tanh")
        self.initial_layer = DenseLayer(layer_width, input_dim, transformation = "tanh")
        
        # define intermediate LSTM layers
        self.n_layers = n_layers
        self.LSTMLayerList = []
                
        for _ in range(self.n_layers):
#             self.LSTMLayerList.append(LSTMLayer(layer_width, input_dim+1))
            self.LSTMLayerList.append(LSTMLayer(layer_width, input_dim))
        
        # define final layer as fully connected with a single output (function value)
        self.final_layer = DenseLayer(1, layer_width, transformation = final_trans)
    
    
    # main function to be called  
    def call(self,X):
        '''            
        Args:
            t: sampled time inputs 
            x: sampled space inputs

        Run the DGM model and obtain fitted function value at the inputs (t,x)                
        '''  
        
        # define input vector as time-space pairs
#         X = tf.concat([t,x],1)

        
        # call initial layer
        S = self.initial_layer.call(X)
        
        # call intermediate LSTM layers
        for i in range(self.n_layers):
            S = self.LSTMLayerList[i].call(S,X)
        
        # call final LSTM layers
        result = self.final_layer.call(S)
        
        return result

In [3]:
delta = tf.constant(0.002, dtype = tf.float64)
phi1 = tf.constant(28.0, dtype = tf.float64)
phi2 = tf.constant(28.0, dtype = tf.float64)
beta1 = tf.constant(0.01, dtype = tf.float64)
beta2 = tf.constant(0.01, dtype = tf.float64)
eta1 = tf.constant(0.012790328319261378, dtype = tf.float64)
eta2 = tf.constant(0.012790328319261378, dtype = tf.float64)
a11 = tf.constant(0.014, dtype = tf.float64)
alpha = tf.constant(0.1, dtype = tf.float64)
scale = np.sqrt(1.754)
sigma_k1 = tf.convert_to_tensor(scale*np.array([[.00477,               .0,   .0]]), dtype = tf.float64)
sigma_k2 = tf.convert_to_tensor(scale*np.array([[.0              , .00477,   .0]]), dtype = tf.float64)
sigma_z =  tf.convert_to_tensor(np.array([[.011*np.sqrt(.5)   , .011*np.sqrt(.5)   , .025]]), dtype = tf.float64)

clowerlim = tf.constant(0.0001, dtype = tf.float64)
rho = tf.constant(1.0, dtype = tf.float64)
gamma = tf.constant(8.0, dtype = tf.float64)
kappa = tf.constant(4.0, dtype = tf.float64)
zeta = tf.constant(0.5, dtype = tf.float64)
wMin = tf.constant(-1., dtype = tf.float64)
wMax = tf.constant(1., dtype = tf.float64)
zMin = tf.constant(-1., dtype = tf.float64)
zMax = tf.constant(1., dtype = tf.float64)


params = {'delta':delta,
         'phi1':phi1,
         'phi2':phi2,
         'beta1':beta1,
         'beta2':beta2,
         'eta1':eta1,
         'eta2':eta2,
         'a11':a11,
         'alpha':alpha,
         'sigma_k1':sigma_k1,
         'sigma_k2':sigma_k2,
         'sigma_z':sigma_z,
         'rho':rho,
         'gamma':gamma,
         'kappa':kappa,
         'zeta':zeta,
         'wMin':wMin,
         'wMax':wMax,
         'zMin':zMin,
         'zMax':zMax,
         'clowerlim':clowerlim}

2023-03-28 15:58:21.433685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 15:58:21.985257: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 324 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:2f:00.0, compute capability: 7.5


In [9]:
@tf.function 
def calder(func,W, Z):
    X = tf.concat([W,Z], axis=1)
    logXiE       = func(X)
    
    dW_logXiE     = tf.gradients(logXiE, W)[0];       dZ_logXiE     = tf.gradients(logXiE, Z)[0];      
    dW2_logXiE    = tf.gradients(dW_logXiE, W)[0];    dZ2_logXiE    = tf.gradients(dZ_logXiE, Z)[0];    
    dWdZ_logXiE   = tf.gradients(dW_logXiE, Z)[0];     
    
    return dW_logXiE, dZ_logXiE, dW2_logXiE, dZ2_logXiE, dWdZ_logXiE

@tf.function 
def calvar(func, W, Z, params):
    X = tf.concat([W,Z], axis=1)
    logXiE       = func(X)
    Vr, Vz, Vrr, Vzz,  Vrz = calder(func, W, Z)

    k1a = (1-params['zeta'] + params['zeta']*tf.exp(W*(1-params['kappa'])))**(1/(params['kappa']-1))
    k2a = ((1-params['zeta'])*tf.exp(W*(params['kappa']-1)) + params['zeta'])**(1/(params['kappa']-1))

    bb1 = (params['alpha'] - 1/params['phi1']*k1a-1/params['phi2']*k2a)
    cc1 = params['delta']*k1a**2/params['phi1']/((1-params['zeta'])*k1a**(1-params['kappa'])-Vr) + params['delta']*k2a**2/params['phi2']/(params['zeta']*(k2a)**(1-params['kappa'])+Vr)
    aa1 = -1
    sqrt_test1 = bb1**2 - 4*aa1*cc1;
    event_A = tf.cast((sqrt_test1 >= 0),tf.float64);

    c_root_large = event_A * (-bb1 - tf.sqrt(event_A*sqrt_test1))/(2*aa1);
    c_root_small = event_A * (-bb1 + tf.sqrt(event_A*sqrt_test1))/(2*aa1);
    c_root_large = c_root_large*tf.cast(tf.math.greater(c_root_large,params['clowerlim']),tf.float64) + \
        params['clowerlim']*tf.ones([batchSize,1],tf.float64)*tf.cast(tf.math.less(c_root_large,params['clowerlim']),tf.float64)

    d1_root = 1/params['phi1']-params['delta']*k1a/((1-params['zeta'])*k1a**(1-params['kappa'])-Vr)/params['phi1']/c_root_large
    d2_root = 1/params['phi2']-params['delta']*k2a/(params['zeta']*(k2a)**(1-params['kappa'])+Vr)/params['phi2']/c_root_large

    uu = params['delta']*tf.math.log(c_root_large)
    
    mu_k1 = (d1_root - params['phi1']/2*d1_root**2) + params['beta1']*Z - eta1
    mu_k2 = (d2_root - params['phi2']/2*d2_root**2) + params['beta2']*Z - eta2
    mu_r = mu_k2 - mu_k1 - 1/2*(tf.reduce_sum(params['sigma_k2']**2) - tf.reduce_sum(params['sigma_k1']**2))

    dkadk1dk1 = (params['kappa']-1)*(1-params['zeta'])**2*(k1a)**(-2*params['kappa']+2) - params['kappa']*(1-params['zeta'])*(k1a)**(-params['kappa']+1)
    dkadk1dk2 = (params['kappa']-1)*params['zeta']*(1-params['zeta'])*(k1a)**(-params['kappa']+1)*(k2a)**(-params['kappa']+1)
    dkadk2dk2 = (params['kappa']-1)*params['zeta']**2*(k2a)**(-2*params['kappa']+2) - params['kappa']*(1-params['zeta'])*(k2a)**(-params['kappa']+1)

    mu_1 = mu_k1*(1-params['zeta'])*(k1a)**(1-params['kappa'])+ \
            mu_k2*(params['zeta'])*(k2a)**(1-params['kappa'])+ \
            1/2*(tf.reduce_sum(params['sigma_k1']**2)*dkadk1dk1 + \
                                                tf.reduce_sum(params['sigma_k2']**2)*dkadk2dk2 + \
                                                2*tf.reduce_sum(params['sigma_k1']*params['sigma_k2'])*dkadk1dk2)

    zdrift = -params['a11']*Z

    h1 = params['sigma_k1'][0,0]*(1-params['zeta'])*(k1a)**(1-params['kappa'])+ params['sigma_k2'][0,0]*(params['zeta'])*(k2a)**(1-params['kappa'])+\
        (params['sigma_k2'][0,0]-params['sigma_k1'][0,0])*Vr + params['sigma_z'][0,0]*Vz
    
    h2 = params['sigma_k1'][0,1]*(1-params['zeta'])*(k1a)**(1-params['kappa'])+ params['sigma_k2'][0,1]*(params['zeta'])*(k2a)**(1-params['kappa'])+\
        (params['sigma_k2'][0,1]-params['sigma_k1'][0,1])*Vr + params['sigma_z'][0,1]*Vz
    
    hz = params['sigma_k1'][0,2]*(1-params['zeta'])*(k1a)**(1-params['kappa'])+ params['sigma_k2'][0,2]*(params['zeta'])*(k2a)**(1-params['kappa'])+\
        (params['sigma_k2'][0,2]-params['sigma_k1'][0,2])*Vr + params['sigma_z'][0,2]*Vz

    penalty_term = (1-params['gamma'])*(h1**2 + h2**2 + hz**2)/2
    uu = uu + penalty_term + mu_1
    t1 = tf.reduce_sum((params['sigma_k2']-params['sigma_k1'])**2)
    t2 = tf.reduce_sum((params['sigma_k2']-params['sigma_k1'])*params['sigma_z'])*2
    t3 = tf.reduce_sum(params['sigma_z']**2)

    higordder = (t1*Vrr + t2*Vrz + t3*Vzz)/2
    
    return logXiE, k1a, k2a, c_root_large, d1_root, d2_root, uu, penalty_term, mu_1, mu_k1, mu_k2, mu_r, zdrift, h1, h2, hz, higordder, Vr, Vz, Vrr, Vzz, Vrz
    
@tf.function 
def HJB_loss_E(func, W, Z, params):
    X = tf.concat([W,Z], axis=1)
    logXiE, k1a, k2a, c_root_large, d1_root, d2_root, uu, penalty_term, mu_1, mu_k1, mu_k2, mu_r, zdrift, h1, h2, hz, higordder, Vr, Vz, Vrr, Vzz, Vrz = calvar(func, W, Z, params)
    HJB = uu + mu_r*Vr + zdrift*Vz + higordder - params['delta']*logXiE
    
    return HJB 

In [12]:
def function_factory(model, loss, valueFunctionLogE, W, Z, params, loss_type, targets):

    ## Obtain the shapes of all trainable parameters in the model
    shapes = tf.shape_n(model.trainable_variables)
    n_tensors = len(shapes)

    # we'll use tf.dynamic_stitch and tf.dynamic_partition later, so we need to prepare required information first
    count = 0
    idx = [] # stitch indices
    part = [] # partition indices

    for i, shape in enumerate(shapes):
        n = np.product(shape)
        idx.append(tf.reshape(tf.range(count, count+n, dtype=tf.int32), shape))
        part.extend([i]*n)
        count += n
    part = tf.constant(part)

    @tf.function
    def assign_new_model_parameters(params_1d):
        params = tf.dynamic_partition(params_1d, part, n_tensors)
        for i, (shape, param) in enumerate(zip(shapes, params)):
            model.trainable_variables[i].assign(tf.reshape(param, shape))

    # Create a function that will compute the value and gradient. This can be the function that the factory returns
    @tf.function
    def val_and_grad(params_1d):
        with tf.GradientTape() as tape:
          ## Update the parameters in the model
            assign_new_model_parameters(params_1d)
            ## Calculate the loss 
            loss_value = loss_type(loss(valueFunctionLogE, W, Z, params), targets)
        ## Calculate gradients and convert to 1D tf.Tensor
        grads = tape.gradient(loss_value, model.trainable_variables)
#         tf.print('grads', grads[-1])
        grads = tf.dynamic_stitch(idx, grads)
        del tape

        ## Print out iteration & loss
        f.iter.assign_add(1)
#         tf.print("Iter:", f.iter, "loss:", loss_value)

        ## Store loss value so we can retrieve later
        tf.py_function(f.history.append, inp=[loss_value], Tout=[])

        return loss_value, grads

    def f(params_1d):
      return [vv.numpy().astype(np.float64)  for vv in val_and_grad(params_1d)]

    ## Store these information as members so we can use them outside the scope
    f.iter = tf.Variable(0)
    f.idx = idx
    f.part = part
    f.shapes = shapes
    f.assign_new_model_parameters = assign_new_model_parameters
    f.history = []

    return f
  

## Training step BFGS
def training_step_BFGS(valueFunctionLogE, W, Z, params, targets, maxiter, maxfun, gtol, maxcor, maxls, ftol):

  ## Train experts NN
  loss_fun = tf.keras.losses.MeanSquaredError()
  func_E = function_factory(valueFunctionLogE, HJB_loss_E, valueFunctionLogE, W, Z, params, loss_fun, targets)
  init_params_E = tf.dynamic_stitch(func_E.idx, valueFunctionLogE.trainable_variables)

  start = time.time()
  results = optimize.minimize(func_E, x0 = init_params_E.numpy(), method = 'L-BFGS-B', jac = True, options = {'maxiter': maxiter, 'maxfun': maxfun, 'gtol': gtol, 'maxcor': maxcor, 'maxls': maxls, 'ftol' : ftol})
  end = time.time()
  print('Elapsed time for experts {:.4f} sec'.format(end - start))
  # after training, the final optimized parameters are still in results.position
  # so we have to manually put them back to the model
  func_E.assign_new_model_parameters(results.x)
 
  return results

In [11]:
points_size = 1
dimension = 2
units = 16
activation = 'tanh'
kernel_initializer = 'glorot_normal'
batchSize = 1024 * points_size
batchSize = 50

iter_num = 50
BFGS_maxiter  = 50
BFGS_maxfun   = 50
BFGS_gtol     = 1.0 * np.finfo(float).eps
BFGS_maxcor   = 100
BFGS_maxls    = 200
BFGS_ftol     = 1.0 * np.finfo(float).eps

## NN structure
tf.keras.backend.set_floatx("float64") ## Use float64 by default

logXiE_NN = tf.keras.Sequential(
    [tf.keras.Input(shape=[dimension,]),
      tf.keras.layers.Dense(units, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dense(units, activation=activation, kernel_initializer=kernel_initializer),
      tf.keras.layers.Dense(1,  activation= None,  kernel_initializer='glorot_normal')])
# logXiE_NN = DGMNet(layer_width =16, n_layers = 3, input_dim = 2, final_trans=None)


start = time.time()
targets = tf.zeros(shape=(batchSize,1), dtype=tf.float64)
for iter in range(iter_num):
  W = tf.random.uniform(shape = (batchSize,1), minval = params['wMin'], maxval = params['wMax'], dtype=tf.float64)
  Z = tf.random.uniform(shape = (batchSize,1), minval = params['zMin'], maxval = params['zMax'], dtype=tf.float64)
#   W = tf.random.normal(shape = (batchSize,1), mean = 0.0, stddev = 0.5, dtype=tf.float64)
#   Z = tf.random.normal(shape = (batchSize,1), mean = 0.0, stddev = 0.5, dtype=tf.float64)
  print('Iteration', iter)
#   results = training_step_BFGS(logXiE_NN, W, Z, params, targets,\
#                     maxiter = BFGS_maxiter, maxfun = BFGS_maxfun, gtol = BFGS_gtol, maxcor = BFGS_maxcor, maxls = BFGS_maxls, ftol = BFGS_ftol)
  print(results.fun)
  if results.fun< 1e-11:
    break
end = time.time()
training_time = '{:.4f}'.format((end - start)/60)
print('Elapsed time for training {:.4f} sec'.format(end - start))

Iteration 0
0.0034132270176228407
Iteration 1
0.0034132270176228407
Iteration 2
0.0034132270176228407
Iteration 3
0.0034132270176228407
Iteration 4
0.0034132270176228407
Iteration 5
0.0034132270176228407
Iteration 6
0.0034132270176228407
Iteration 7
0.0034132270176228407
Iteration 8
0.0034132270176228407
Iteration 9
0.0034132270176228407
Iteration 10
0.0034132270176228407
Iteration 11
0.0034132270176228407
Iteration 12
0.0034132270176228407
Iteration 13
0.0034132270176228407
Iteration 14
0.0034132270176228407
Iteration 15
0.0034132270176228407
Iteration 16
0.0034132270176228407
Iteration 17
0.0034132270176228407
Iteration 18
0.0034132270176228407
Iteration 19
0.0034132270176228407
Iteration 20
0.0034132270176228407
Iteration 21
0.0034132270176228407
Iteration 22
0.0034132270176228407
Iteration 23
0.0034132270176228407
Iteration 24
0.0034132270176228407
Iteration 25
0.0034132270176228407
Iteration 26
0.0034132270176228407
Iteration 27
0.0034132270176228407
Iteration 28
0.003413227017622

In [46]:
@tf.function
def lossfun():
    loss_type = tf.keras.losses.MeanSquaredError()
#     loss = loss_type(HJB_loss_E(logXiE_NN, W, Z, params), targets)
    loss = tf.reduce_mean(tf.square(HJB_loss_E(logXiE_NN, W, Z, params)))
    return loss

In [42]:
loss.name

AttributeError: Tensor.name is undefined when eager execution is enabled.

In [47]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
with tf.GradientTape() as tp:
    loss = lossfun()
optimizer.minimize(lossfun(), var_list=[W, Z],tape=tf.GradientTape())

AttributeError: Tensor.name is undefined when eager execution is enabled.

In [30]:
lossfun(W, Z)

<tf.Tensor: shape=(), dtype=float64, numpy=19262.111866882613>

In [28]:
lossfun(W, Z)

<tf.Tensor: shape=(), dtype=float64, numpy=19262.111866882613>